In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [2]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'd-pbv-model'
id_name = 'activity_id'
target = 'outcome'

file_name = "./d-pbv/test_Predicting Red Hat Business Value.csv"
test_file_name = "./d-pbv/test_Predicting Red Hat Business Value-no-label.csv"

kaggle_file_name = "./d-pbv/kaggle-test-Predicting Red Hat Business Value.csv"
kaggle_test_file_name = "./d-pbv/kaggle-test-Predicting Red Hat Business Value-no-label.csv"

kaggle_pred_file_name = "./d-pbv/kaggle-test-Predicting Red Hat Business Value-predictions.csv"

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

    people_id   activity_id        date activity_category  char_1   char_2  \
0   ppl_64887     act2_1054  2022-09-22            type 3     NaN      NaN   
1  ppl_305838  act2_1493887  2023-02-15            type 2     NaN      NaN   
2     ppl_185     act1_8923  2023-08-26            type 1  type 5   type 2   
3  ppl_166168   act1_282097  2022-10-20            type 1  type 3  type 10   
4  ppl_135399   act2_446102  2023-06-08            type 2     NaN      NaN   

   char_3  char_4  char_5  char_6  char_7  char_8  char_9  char_10  
0     NaN     NaN     NaN     NaN     NaN     NaN     NaN  type 23  
1     NaN     NaN     NaN     NaN     NaN     NaN     NaN   type 1  
2  type 7  type 3  type 1  type 3  type 7  type 4  type 7      NaN  
3  type 5  type 1  type 6  type 1  type 1  type 5  type 5      NaN  
4     NaN     NaN     NaN     NaN     NaN     NaN     NaN   type 1  
(219730, 14)


In [13]:
f = open(test_file_name)

count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)    
    count = count + 1


In [15]:
print(pred_label[:10])
print(pred_neg_prpb[:10])
print(pred_pos_prpb[:10])

import pandas as pd
import numpy as np

pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print(np.mean(y_true==y_pred))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_pos_prpb, pos_label=1)
metrics.auc(fpr, tpr)

[1, 1, 0, 0, 0, 1, 1, 0, 0, 0]
[2.6941299438476562e-05, 0.44138211011886597, 0.5522752702236176, 0.5780945718288422, 0.5124545097351074, 0.4987850785255432, 0.46560537815093994, 0.8862053826451302, 0.5006422996520996, 0.5410961210727692]
[0.9999730587005615, 0.558617889881134, 0.44772472977638245, 0.42190542817115784, 0.4875454902648926, 0.5012149214744568, 0.5343946218490601, 0.11379461735486984, 0.4993577003479004, 0.45890387892723083]
0.6387657579756975


0.7166628807934289

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

#df = df.drop([id_name],axis=1) 

print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

    people_id   activity_id        date activity_category   char_1   char_2  \
0  ppl_100004   act1_249281  2022-07-20            type 1   type 5  type 10   
1  ppl_100004   act2_230855  2022-07-20            type 5      NaN      NaN   
2   ppl_10001   act1_240724  2022-10-14            type 1  type 12   type 1   
3   ppl_10001    act1_83552  2022-11-27            type 1  type 20  type 10   
4   ppl_10001  act2_1043301  2022-10-15            type 5      NaN      NaN   

   char_3  char_4  char_5  char_6  char_7   char_8   char_9    char_10  
0  type 5  type 1  type 6  type 1  type 1   type 7   type 4        NaN  
1     NaN     NaN     NaN     NaN     NaN      NaN      NaN   type 682  
2  type 5  type 4  type 6  type 1  type 1  type 13  type 10        NaN  
3  type 5  type 4  type 6  type 1  type 1   type 5   type 5        NaN  
4     NaN     NaN     NaN     NaN     NaN      NaN      NaN  type 3015  
(498687, 14)


In [ ]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [7]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

    activity_id  outcome
0   act1_249281        0
1   act2_230855        0
2   act1_240724        1
3    act1_83552        1
4  act2_1043301        1
(498687, 2)


In [8]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '016f2ea9-f522-44a3-a53f-3185a7d70aba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '016f2ea9-f522-44a3-a53f-3185a7d70aba',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:47:54 GMT'},
  'RetryAttempts': 0}}